In [1]:
import pandas as pd
import numpy as np
import rpy2.robjects as ro
from rpy2.robjects import r as R
from rpy2.robjects import pandas2ri
import sqlite3 as sql
import pandas.io.sql as pdsql
import re

In [2]:
db = "/ifs/projects/proj052/pipeline_proj052/csvdb"
dbh = sql.connect(db)
pandas2ri.activate()

# have to pull out arrays in R
# need to unserialize in R as well, Python doesn't reconstruct the arrays properly
R('''library(RSQLite)''')
R('''con <- dbConnect(SQLite(), "%s")''' % db)
py_sqlcon = R["con"]

/ifs/apps/apps/python-2.7.10/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: DBI

  res = super(Function, self).__call__(*new_args, **new_kwargs)


In [3]:
print py_sqlcon.__class__

<class 'rpy2.robjects.methods.RS4'>


In [4]:
def setupConnection(db_path):
    '''
    setup an SQLite connection in R, then return it as an rpy2 object'''
    
    R('''require(RSQLite)''')
    R('''con <- dbConnect(SQLite(), "%s")''' % db_path)
    py_sqlcon = R["con"]
    
    return py_sqlcon    

In [5]:
r_con = setupConnection(db)
R.assign("pycon", r_con)
R('''dbDisconnect(pycon)''')

<Vector - Python:0x2afede9885a8 / R:0x2e7ccd8>
[       1]

In [6]:
variable = [("gate", "TEXT"), ("batch", "TEXT"), ("rows", "TEXT")]
", ".join([" ".join([x, y]) for x,y in variable])

'gate TEXT, batch TEXT, rows TEXT'

In [7]:
R('''df <- data.frame(replicate(5, rnorm(20)))''')
R('''colnames(df) <- c("var1", "var2", "var3", "var4", "var5")''')
R('''cols <- colnames(df)''')
df_headers = [x for x in R["cols"]]

In [8]:
variables = ["str1", "str2", "str3", "str4", "str5"]

In [9]:
zip(df_headers, df_headers)

[('var1', 'var1'),
 ('var2', 'var2'),
 ('var3', 'var3'),
 ('var4', 'var4'),
 ('var5', 'var5')]

In [3]:
db = "/ifs/projects/proj052/pipeline_proj052/csvdb"
R('''library(RSQLite)''')
#R('''con <- dbConnect(SQLite(), "%s")''' % db)
#py_con = R["con"]

dbh = sql.connect(db)

In [4]:
# get all unique combinations of panel and gate
query = "SELECT gate, batch, panel FROM CD8_Tcell_fano;"
#R('''dbGetPreparedQuery(con, "%s")''' % query)
records = pdsql.read_sql(query, dbh)

In [5]:
gates = set(zip(records["gate"], records["panel"]))

In [47]:
# Filter arrays? If sum of all elements is 0, then drop it?
R('''source("/ifs/projects/proj052/src/flow_pipeline/R_scripts/FlowProcess.R")''')
R('''library(data.table)''')
R('''library(rbenchmark)''')
for gate, panel in gates:
    if panel == "P2":
        pass
    else:
        state = '''SELECT * FROM CD8_Tcell_fano WHERE gate = '%s' AND panel = '%s' ''' % (gate, "P1")
        R('''r.df <- dbGetQuery(con, "%(state)s")''' % locals())
        R('''r.df$real.array <- lapply(unlist(r.df$array), FUN=function(x) {unserialize(charToRaw(x))})''')
        # need to add batch to each array - write a function
        R('''batch_list <- add_batch(r.df)''')
        R('''out_df <- r.df[,c(1:4,6,7)]''')
        #R('''print(lapply(r.df$real.array, FUN=colnames))''')
        #R('''arrays <- data.frame(do.call(rbind, r.df$real.array))''')
        R('''arrays <- data.frame(rbindlist(r.df$real.array))''')
        #R('''meta.df <- r.df[,c(1:4,6)]''')
        R('''twins <- unlist(strsplit(r.df$rows, fixed=T, split="/"))''')
        R('''arrays$twins <- twins''')
        R('''arrays$batch <- unlist(batch_list)''')
        R('''arrays$panel <- "%(panel)s"''' % locals())
        R('''arrays$gate <- "%(gate)s"''' % locals())
        # remove NA columns
        R('''cols <- colnames(arrays)[!is.na(colnames(arrays))]''')
        R('''all_cols <- cols[!grepl("NA", cols, perl=TRUE)]''')
        R('''print(head(arrays[all_cols]))''')
        py_df = R["arrays"]
        #py_meta = R["meta.df"]
        #_meta = pandas2ri.ri2py_dataframe(py_meta)
        #_df = pandas2ri.ri2py_dataframe(py_df)
        #print _df
        break

          CD27      CD127       Aq       CD8      CD4     CD57   CD45RA
1 0.0003982081 0.04187760 7.392269 11.369756 12.53314 1.707251 1.597268
2 0.0003131674 0.04800354 9.092283 10.789068 14.27233 2.173279 1.704395
3 0.0003978960 0.04312275 7.919096 10.663936 13.25227 2.069882 1.621518
4 0.0003805075 0.04255617 8.894760 10.223513 13.88786 2.247048 1.405900
5 0.0003488279 0.04245987 8.887288  8.828831 14.66470 2.139274 1.594047
6 0.0003811960 0.04569761 8.281769  7.716811 14.87854 2.554836 1.758243
       CCR7      CD3     CD95      CD28     CD244     CD31        twins batch
1 0.2209000 1.238972 4.305136 0.7535486 13.079434 2.276457  Twin_TS01_1  ZZEV
2 0.4531488 2.193631 4.912340 0.8438717 11.483903 2.360154 Twin_TS01_10  ZZEV
3 0.4222162 1.425359 4.263241 0.6738846 11.071193 2.103901 Twin_TS01_11  ZZEV
4 0.3172391 1.427968 4.786332 0.6811353 12.886009 1.921735 Twin_TS01_12  ZZEV
5 0.2536503 2.144931 3.028471 0.8592804  9.863618 1.767836 Twin_TS01_13  ZZEV
6 0.9852331 3.459234 2.83239